In [1]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np 

from sklearn.metrics import mean_absolute_error
import tensorflow_addons as tfa

c:\Users\21265\anaconda3\envs\tfpy39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### Load data

In [3]:
train_ds = tf.data.experimental.load('./data/audio/train_ds/') \
    .cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)


valid_ds = tf.data.experimental.load('./data/audio/valid_ds/') \
    .cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [4]:
train_ds, valid_ds

(<PrefetchDataset shapes: ((None, 15, 128), (None, 5)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 15, 128), (None, 5)), types: (tf.float32, tf.float32)>)

### Build Audio model

In [2]:
def make_audio_model():
  inputs = keras.layers.Input(shape=(15,128))

  x = keras.layers.Conv1D(32, 2)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  
  x = keras.layers.Conv1D(64, 2)(x)
  x = keras.layers.Dropout(0.3)(x)

  x = keras.layers.LSTM(512, return_sequences=True)(x)
  x = keras.layers.LSTM(256)(x)

  x = keras.layers.Dense(256)(x)
  x = keras.layers.Dropout(0.3)(x)


  x = keras.layers.Dense(5, activation='sigmoid')(x)

  return keras.models.Model(inputs=inputs, outputs=x, name='audio_model')


audio_model = make_audio_model()


### Compile model

In [ ]:
import datetime
t = datetime.datetime.now().strftime("%m%d_%H%M%S")

optimizer   = tfa.optimizers.RectifiedAdam()

early_stopping = keras.callbacks.EarlyStopping(patience=10, verbose=0)
check_point    = keras.callbacks.ModelCheckpoint(filepath='./weights/audio/'+str(t)+'/audio.t5',
                             monitor='val_mae',
                             mode='min',
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=0)

audio_model.compile(loss='mse', optimizer=optimizer , metrics=['mae'])

### Train

In [35]:
history = audio_model.fit(train_ds, validation_data=valid_ds, batch_size=32, epochs=100, callbacks=[early_stopping, check_point])

Epoch 1/100
188/188 [==============================] - 9s 35ms/step - loss: 0.0194 - mae: 0.1113 - val_loss: 0.0163 - val_mae: 0.1020
Epoch 2/100
188/188 [==============================] - 5s 24ms/step - loss: 0.0177 - mae: 0.1061 - val_loss: 0.0161 - val_mae: 0.1009
Epoch 3/100
188/188 [==============================] - 5s 25ms/step - loss: 0.0174 - mae: 0.1053 - val_loss: 0.0165 - val_mae: 0.1023
Epoch 4/100
188/188 [==============================] - 5s 26ms/step - loss: 0.0174 - mae: 0.1052 - val_loss: 0.0174 - val_mae: 0.1055
Epoch 5/100
188/188 [==============================] - 5s 26ms/step - loss: 0.0171 - mae: 0.1043 - val_loss: 0.0158 - val_mae: 0.0998
Epoch 6/100
188/188 [==============================] - 5s 25ms/step - loss: 0.0170 - mae: 0.1040 - val_loss: 0.0178 - val_mae: 0.1053
Epoch 7/100
188/188 [==============================] - 5s 26ms/step - loss: 0.0168 - mae: 0.1034 - val_loss: 0.0158 - val_mae: 0.0999
Epoch 8/100
188/188 [==============================] - 5s 26ms

### Load weights

In [3]:
audio_model.load_weights('./weights/audio/0225_191323_9005/audio.t5')

## Evaluation

### Validation data

In [5]:
valid_ds = tf.data.experimental.load('./data/audio/valid_ds')

y_true = np.concatenate([y for x,y in valid_ds])
y_pred = audio_model.predict(valid_ds)

mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
(1-mae)*100, (1-np.mean(mae))*100

(array([90.441124, 89.76602 , 89.94365 , 90.76098 , 90.0224  ],
       dtype=float32),
 90.18683582544327)

### Test data

In [6]:
test_ds = tf.data.experimental.load('./data/audio/test_ds/')
loss, mae = audio_model.evaluate(test_ds)


y_true = np.concatenate([y for x,y in test_ds])
y_pred = audio_model.predict(test_ds)

mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
(1-mae)*100, (1-np.mean(mae))*100

63/63 [==============================] - 1s 7ms/step - loss: 0.0156 - mae: 0.0994


(array([90.31917 , 89.90838 , 89.861176, 90.41583 , 89.78329 ],
       dtype=float32),
 90.05756974220276)